# Movielens 영화 추천
- 별점을 시청횟수로 가정 
- 3점 미만은 선호하지 않는다고 가정

In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'counts', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.shape

(1000209, 4)

In [2]:
len(ratings.counts)
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['counts']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 countS로 바꿉니다.
# ratings.rename(columns={'rating':'count'}, inplace=True)
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 제목 소문자화 및 연도 제외 및 공백 제거

In [6]:
movies['title'] = movies['title'].str.lower()
movies['title'] = movies['title'].str.split(pat='(').map(lambda x: x[0])
movies['title'] = movies['title'].str.split().agg("".join)

In [7]:
movies['title']

0                     toystory
1                      jumanji
2               grumpieroldmen
3              waitingtoexhale
4       fatherofthebridepartii
                 ...          
3878            meettheparents
3879          requiemforadream
3880                 tigerland
3881            twofamilyhouse
3882             contender,the
Name: title, Length: 3883, dtype: object

# 2.분석하기
- ratings에 있는 유니크한 영화 갯수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기수)

In [8]:
print('ratings에 있는 유니크한 영화 갯수',ratings['movie_id'].nunique())
print('ratings에 있는 유니크한 사용자 수',ratings['user_id'].nunique())
movies_count = ratings.groupby('movie_id')['user_id'].count()
# movies_count.sort_values(ascending=False).head(30)
ratings['movie_id'] = ratings['movie_id'].replace(movies['title'])
# 영화의 겹치는 것이 있는지 동일제목 장르 등
print(sum(movies.duplicated()))



ratings에 있는 유니크한 영화 갯수 3628
ratings에 있는 유니크한 사용자 수 6039
0


In [9]:
# 가장 인기 있는 영화 30개
movies_count = ratings.groupby('movie_id')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)


movie_id
briefencounter                 3211
ladybirdladybird               2910
alien                          2885
ragingbull                     2716
somethingwickedthiswaycomes    2561
silenceofthelambs,the          2509
prettywoman                    2498
bigblue,the                    2473
somekindofwonderful            2460
superman                       2434
lassie                         2413
dogofflanders,a                2385
pallbearer,the                 2371
rumbleinthebronx               2314
wishmaster                     2297
secretgarden,the               2257
armyofdarkness                 2252
deadlyfriend                   2213
stripes                        2210
strawberryandchocolate         2194
kansascity                     2167
associate,the                  2157
heathers                       2121
tora!tora!tora!                2066
talesofterror                  2051
priest                         2030
ilovetrouble                   2022
m                  

# 3.선호하는 영화 5가지 골라 추가하기 

In [10]:
data = ratings[['user_id','movie_id','counts']]

In [11]:
my_lovemv = ['toystory','jumanji','grumpieroldmen','superman','priest']
my_playlist = pd.DataFrame({'user_id': ['hsc']*5, 'movie_id': my_lovemv, 'counts':[5]*5})

if not data.isin({'user_id':['hsc']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts
1000203,6040,"leopardson,the",3
1000205,6040,birdofprey,5
1000206,6040,nakedinnewyork,5
1000207,6040,palookaville,4
1000208,6040,"associate,the",4
0,hsc,toystory,5
1,hsc,jumanji,5
2,hsc,grumpieroldmen,5
3,hsc,superman,5
4,hsc,priest,5


In [12]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
print(user_to_idx['hsc'])
print(movie_to_idx['alien'])

6039
117


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,3585,5
1,6039,40,5
2,6039,511,5
3,6039,124,5


# 4.CSR matrix 직접만들기

In [16]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,wingsofdesire,5,978300760
1,1,bloodsport2,3,978302109
2,1,allabouteve,3,978301968
3,1,empirerecords,4,978300275
4,1,you'vegotmail,5,978824291


In [17]:
data.head()

,user_id,movie_id,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5


In [18]:

from scipy.sparse import csr_matrix
num_user = data['user_id'].nunique()
num_artist = data['movie_id'].nunique()

# csr_data = csr_matrix((data.counts, (data.user_id, data.movie_id)))
csr_data = csr_matrix((data.counts, (data.user_id, data.movie_id)), shape= (num_user, num_artist))
csr_data

<6040x3586 sparse matrix of type '<class 'numpy.int64'>'
	with 835559 stored elements in Compressed Sparse Row format>

# 5. als_model = AlternatinngLeastSquares 모델 구성하여 훈련

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [27]:
als_model = AlternatingLeastSquares(factors=512, regularization=0.01, iterations=20, dtype=np.float32)

In [28]:
csr_data_transpose = csr_data.T
csr_data_transposecsr_data_transpose = csr_data.T


In [29]:
als_model.fit(csr_data_transpose)

100%|██████████| 20/20 [00:07<00:00,  2.79it/s]


# 6.내가 선호하는 5가지 영화 중 하나와 그 외의 영화를 하나를 골라 훈련된 모델이 예측한 선호도 파악

In [30]:
def test(x):
    hsc = user_to_idx['hsc']
    hsc_vector = als_model.user_factors[hsc]
    a = movie_to_idx[x]
    a_vec = als_model.item_factors[a]
    return np.dot(hsc_vector, a_vec)
test('jumanji')


0.9427357

In [31]:
test('alien')


0.05518607

음 변명을 하자면 영화가 뭔지 잘 모르고 넣어놔서 좋아하는 영화가 일단 저기에 없었다... 그래서 예측을 할 수 있는지 테스트가 안된다...

# 7. 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [35]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

favorite_movie = 'superman'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)

['superman',
 'toystory',
 'fatalbeauty',
 'eraser',
 'forevermozart',
 'beautifulpeople',
 'lateaugust,earlyseptember',
 'airport1975',
 'girl6',
 'funeral,the',
 '7thvoyageofsinbad,the',
 'diebinnen',
 'indianinthecupboard,the',
 'catballou',
 'dogdayafternoon']

# 8. 내가 가장 좋아할 만한 영화들 추천받기